In [9]:

import tensorflow as tf
import os
import numpy as np
import random
import cv2 as cv




import matplotlib.pyplot as plt

##from skimage.io import imread,imshow

# model = tf.keras.models.load_model('Modelv1',compile=False)

# data = np.load('data.npz')
# X_train = data['X_train']
# X_test = data['X_test']
# Y_test = data['Y_test']

# preds_test = model.predict(X_test)
#preds_test = (preds_test > 0.5).astype(np.uint8)












2023-05-05 19:31:21.644424: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
preds_test = (preds_test > 0.5).astype(np.uint8)
1 in preds_test[0]

NameError: name 'preds_test' is not defined

In [ ]:
ix = random.randint(0, len(X_test))
img = cv.cvtColor(X_test[1],cv.COLOR_BGR2GRAY)
img2=np.array(preds_test[1]).astype('uint8')

# img2 = img2.transpose(2,0,1).reshape(128,-1)


for i in range(img2.shape[0]):
    for j in range(img2.shape[1]):
        if img2[i][j] == 1:
            img2[i][j] = 255


cv.imshow('X',img)
cv.waitKey(0)
# cv.imshow('Y',np.array(Y_test[ix]).astype('uint8'))
cv.imshow('Pred',img2)





cv.waitKey(0)
cv.destroyAllWindows()



NameError: name 'X_test' is not defined

In [8]:
img3 = cv.imread('/home/omkar/FireClassifier/segmented_imgs/img/img75/Fire2.jpg')
# img3 = cv.imread('/home/omkar/FireClassifier/Flame/Flame0.jpg')
img4 = img3[:,:,2]
img5 = cv.blur(img4,(10,10))
contours, hierarchy = cv.findContours(img4,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
contours2, hierarchy = cv.findContours(img5,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
print(len(contours),len(contours2))

194 2


In [9]:
c1 = contours2
# c = list(c[5])
# c_sort_by_y = sorted(c,key = lambda x : x[0][1])
# c_sort_by_x = sorted(c,key = lambda x : x[0][0])
# c_sort_by_y
c_selected = c1[0]
for i in c1:
    if len(c_selected)< len(i): 
        c_selected = i

c = list(c_selected)
c_sort_by_y = sorted(c,key = lambda x : x[0][1])
c_sort_by_x = sorted(c,key = lambda x : x[0][0])
print(cv.contourArea(c_selected),len(c_selected))

159750.0 1006


In [28]:

import math

fire_img1=cv.imread('/home/omkar/FireClassifier/segmented_imgs/img/img75/Fire2.jpg')
fire_img2=cv.imread('/home/omkar/FireClassifier/segmented_imgs/img/img75/Fire19.jpg')
flame_img1=cv.imread('/home/omkar/FireClassifier/segmented_imgs/img/Fire/Flame11.jpg')
flame_img2=cv.imread('/home/omkar/FireClassifier/segmented_imgs/img/Fire/Flame1.jpg')

def contourprep(img):
    img = img[:,:,2]
    img = cv.blur(img,(10,10))
    contours, hierarchy = cv.findContours(img,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    c_selected = contours[0]
    for i in contours:
        if len(c_selected)< len(i): 
            c_selected = i

    c = list(c_selected)
    c_sort_by_y = sorted(c,key = lambda x : x[0][1])
    c_sort_by_x = sorted(c,key = lambda x : x[0][0])
    return contours,c_sort_by_x,c_sort_by_y
    
    

    
def getBaseWidthAndHeight(arr:list):
    min_y = arr[len(arr)-1][0][1]
    max_y = arr[0][0][1]
    range = (min_y - max_y)/4
    min_x = arr[len(arr)-1][0][0]
    max_x = arr[len(arr)-1][0][0]
    #print('hehe',max_x,min_x,range,max_y,min_y,min_y-range)

    
    i = len(arr)-1
    while i >= 0:
        if(arr[i][0][1] < int(min_y-range)):
            break
        if(arr[i][0][0] > max_x):
            max_x = arr[i][0][0]
        if(arr[i][0][0] < min_x):
            min_x = arr[i][0][0]
        i = i -1
    midpoint = math.floor((min_x + max_x)/2)

    return {"width":max_x - min_x,"height":min_y-max_y,"base_midpoint":midpoint}



def getSymmetryScore(arr:list,base_midpoint):
    

    min_x = arr[len(arr)-1][0][0]
    max_x = arr[0][0][0]
    mean = base_midpoint
    left_area = 0
    right_area = 0

    for i in range(0,len(arr)):
        if arr[i][0][0] < mean:
            left_area += mean-arr[i][0][0]
        else:
            right_area += arr[i][0][0]-mean
    return abs(left_area-right_area)/(left_area+right_area)

def getAttrributes(img):
    h_mat = img[:,:,0]
    s_mat = img[:,:,1]
    v_mat = img[:,:,2]
    number_of_nonblackpixels = np.count_nonzero(v_mat)
    average_h = h_mat[np.nonzero(h_mat)].mean()
    average_s = s_mat[np.nonzero(s_mat)].mean()
    average_v = v_mat[np.nonzero(v_mat)].mean()
    return number_of_nonblackpixels,average_h,average_s,average_v

def getArclengthAreaRatio(img):
    number_of_nonblackpixels = np.count_nonzero(img[:,:,2])
    contours, hierarchy = cv.findContours(img[:,:,2],cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    total_arclength = 0
    for i in contours:
        total_arclength += cv.arcLength(i,True)

    return total_arclength

def whitePixelsAreaRatio(img):
    
    number_of_nonblackpixels = np.count_nonzero(img[:,:,2])
    return len(np.where(img[:,:,2]>250))/number_of_nonblackpixels

# countours,c_sort_by_x,c_sort_by_y=contourprep(img3)





# countours,c_sort_by_x,c_sort_by_y=contourprep(cv_image)
#basewidth_Height_base_midpoint = getBaseWidthAndHeight(c_sort_by_y)
#print(basewidth_Height_base_midpoint,getSymmetryScore(c_sort_by_x,basewidth_Height_base_midpoint["base_midpoint"]))
print('Fireimg1',getArclengthAreaRatio(fire_img1))
print('Flameimg1',getArclengthAreaRatio(flame_img1))
print('Fireimg2',getArclengthAreaRatio(fire_img2))
print('Flameimg2',getArclengthAreaRatio(flame_img2))

#basewidth_Height_base_midpoint = getBaseWidthAndHeight(c_sort_by_y)
#print(fire,getSymmetryScore(c_sort_by_x,fire["base_midpoint"]))
#print('\n')
#print(getArclengthAreaRatio(img3))
#print('\n')

#getSymmetryScore(c_sort_by_x,fire["base_midpoint"])
# img3 = img3.astype(np.uint8)
# img3 = cv.drawContours(img3, c, -1, (0,255,0), 3)



Fireimg1 8442.307565569878
Flameimg1 2208.2022211551666
Fireimg2 22235.900946497917
Flameimg2 7982.573382258415


In [ ]:
print(np.count_nonzero(img3),img3.shape[0]*img3.shape[1])


443341 518400


In [11]:
cv.imshow('Pred',img4)





cv.waitKey(0)
cv.destroyAllWindows()

In [17]:
import re
import os
import numpy as np
import cv2 as cv

import math
folder='/home/omkar/FireClassifier/segmented_imgs/img/Fire'
images=[]
filename1 = ''
i=0





def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

def contourprep(img):
    img = img[:,:,2]
    img = cv.blur(img,(10,10))
    contours, hierarchy = cv.findContours(img,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    c_selected = contours[0]
    for i in contours:
        if len(c_selected)< len(i): 
            c_selected = i

    c = list(c_selected)
    c_sort_by_y = sorted(c,key = lambda x : x[0][1])
    c_sort_by_x = sorted(c,key = lambda x : x[0][0])
    return contours,c_sort_by_x,c_sort_by_y
    
    

    
def getBaseWidthAndHeight(arr:list):
    min_y = arr[len(arr)-1][0][1]
    max_y = arr[0][0][1]
    range = (min_y - max_y)/4
    min_x = arr[len(arr)-1][0][0]
    max_x = arr[len(arr)-1][0][0]
    #print('hehe',max_x,min_x,range,max_y,min_y,min_y-range)

    
    i = len(arr)-1
    while i >= 0:
        if(arr[i][0][1] < int(min_y-range)):
            break
        if(arr[i][0][0] > max_x):
            max_x = arr[i][0][0]
        if(arr[i][0][0] < min_x):
            min_x = arr[i][0][0]
        i = i -1
    midpoint = math.floor((min_x + max_x)/2)

    return {"width":max_x - min_x,"height":min_y-max_y,"base_midpoint":midpoint}



def getSymmetryScore(arr:list,base_midpoint):
    

    min_x = arr[len(arr)-1][0][0]
    max_x = arr[0][0][0]
    mean = base_midpoint
    left_area = 0
    right_area = 0

    for i in range(0,len(arr)):
        if arr[i][0][0] < mean:
            left_area += mean-arr[i][0][0]
        else:
            right_area += arr[i][0][0]-mean
    return abs(left_area-right_area)/(left_area+right_area)

def getAttrributes(img):
    h_mat = img[:,:,0]
    s_mat = img[:,:,1]
    v_mat = img[:,:,2]
    number_of_nonblackpixels = np.count_nonzero(v_mat)
    average_h = h_mat[np.nonzero(h_mat)].mean()
    average_s = s_mat[np.nonzero(s_mat)].mean()
    average_v = v_mat[np.nonzero(v_mat)].mean()
    return number_of_nonblackpixels,average_h,average_s,average_v

def getArclengthAreaRatio(img):
    number_of_nonblackpixels = np.count_nonzero(img[:,:,2])
    contours, hierarchy = cv.findContours(img[:,:,2],cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    total_arclength = 0
    for i in contours:
        total_arclength += cv.arcLength(i,True)

    return total_arclength/number_of_nonblackpixels

def whitePixelsAreaRatio(img):
    
    number_of_nonblackpixels = np.count_nonzero(img[:,:,2])
    return len(np.where(img[:,:,2]>250))/number_of_nonblackpixels




non_masked_images = os.listdir(folder)
non_masked_images = sorted_alphanumeric(non_masked_images)
for img in non_masked_images:
        
        cv_image = cv.imread(folder+"/"+img)
        if cv_image is not None:
            countours,c_sort_by_x,c_sort_by_y=contourprep(cv_image)
            basewidth_Height_base_midpoint = getBaseWidthAndHeight(c_sort_by_y)
            print(basewidth_Height_base_midpoint,getSymmetryScore(c_sort_by_x,basewidth_Height_base_midpoint["base_midpoint"]))

            print(getArclengthAreaRatio(cv_image))

                
            #cv.imwrite(filename1+"Flame" + str(i)+ ".jpg", output)
            i=i+1

 14%|█▍        | 5/36 [00:00<00:00, 48.27it/s]

{'width': 750, 'height': 373, 'base_midpoint': 476} 0.2316798767509274
0.06419028811669683
{'width': 577, 'height': 466, 'base_midpoint': 449} 0.4306223076222741
0.09042281088028258
{'width': 959, 'height': 539, 'base_midpoint': 479} 0.0010427528675703858
0.008034990619406067
{'width': 959, 'height': 411, 'base_midpoint': 479} 0.03351104341203351
0.054404252098416504
{'width': 959, 'height': 425, 'base_midpoint': 479} 0.03141375772867164
0.04816821721882144
{'width': 540, 'height': 352, 'base_midpoint': 622} 0.2010226580530032
0.14183871549258553
{'width': 959, 'height': 539, 'base_midpoint': 479} 0.0010427528675703858
0.008023871264136107
{'width': 959, 'height': 422, 'base_midpoint': 479} 0.9263204725771769
0.03364635435407984
{'width': 959, 'height': 458, 'base_midpoint': 479} 0.17385828424491295
0.08685857315316028
{'width': 497, 'height': 360, 'base_midpoint': 483} 0.11470396491436022
0.1049975957116487
{'width': 519, 'height': 267, 'base_midpoint': 407} 0.1778521834910649


 44%|████▍     | 16/36 [00:00<00:00, 47.96it/s]

0.09098494911167553
{'width': 111, 'height': 257, 'base_midpoint': 479} 0.08571428571428572
0.06823523216075321
{'width': 227, 'height': 152, 'base_midpoint': 535} 0.03991277340910593
0.09347653664592888
{'width': 86, 'height': 244, 'base_midpoint': 134} 0.541258557997838
0.23739486027554418
{'width': 305, 'height': 276, 'base_midpoint': 647} 0.025544954742683507
0.13036238379318973
{'width': 365, 'height': 300, 'base_midpoint': 664} 0.5754056838670902
0.12127785591167764
{'width': 464, 'height': 403, 'base_midpoint': 501} 0.18725848067250872
0.052685122468132346
{'width': 365, 'height': 329, 'base_midpoint': 584} 0.003948782653395018
0.057210891930702924
{'width': 555, 'height': 224, 'base_midpoint': 681} 0.045959028915352196
0.09107868178705993
{'width': 227, 'height': 407, 'base_midpoint': 458} 0.11983896495029119


 72%|███████▏  | 26/36 [00:00<00:00, 44.86it/s]

0.04798023239906696
{'width': 175, 'height': 347, 'base_midpoint': 457} 0.405534186807509
0.054781961319157496
{'width': 959, 'height': 424, 'base_midpoint': 479} 0.5595703581907989
0.19373086100819997
{'width': 959, 'height': 358, 'base_midpoint': 479} 0.945305351397621
0.031901835927764584
{'width': 383, 'height': 289, 'base_midpoint': 191} 0.822933557625042
0.0873991951771266
{'width': 762, 'height': 401, 'base_midpoint': 381} 0.35548080255927994
0.10678096684156886
{'width': 232, 'height': 415, 'base_midpoint': 483} 0.16726355611601512
0.04750278880327988
{'width': 238, 'height': 195, 'base_midpoint': 719} 0.8342001576044129
0.05831698254073244
{'width': 959, 'height': 348, 'base_midpoint': 479} 0.1377586370861467
0.1572497019113991
{'width': 264, 'height': 366, 'base_midpoint': 487} 0.009339460236129749
0.04287157610595832
{'width': 399, 'height': 205, 'base_midpoint': 471} 0.10390101391991752
0.06183605025720864


100%|██████████| 36/36 [00:00<00:00, 49.59it/s]

{'width': 959, 'height': 45, 'base_midpoint': 479} 0.2938876639465369
0.14442154467169943
{'width': 295, 'height': 262, 'base_midpoint': 629} 0.25074249115467057
0.10919379198355361
{'width': 959, 'height': 45, 'base_midpoint': 479} 0.12979144683020524
0.13435886734520325
{'width': 319, 'height': 399, 'base_midpoint': 684} 0.1356570326739432
0.17230997394141062
{'width': 175, 'height': 176, 'base_midpoint': 470} 0.0280643026740882
0.13904584896601208
{'width': 959, 'height': 45, 'base_midpoint': 479} 0.28753892101417045
0.15830875873211533
